In [ ]:
# !pip install langchain
# !pip install langchain-community
# !pip install openai
# !pip install unstructured
# !pip install "unstructured[docx]"

In [ ]:
import requests
from bs4 import BeautifulSoup  # HTML 태그 제거용 라이브러리
import re  # 특수 문자 제거용 모듈

class McDonaldsMenu:
    def __init__(self):
        # 메뉴 데이터를 가져올 API의 URL과 헤더 설정
        self.url = "https://www.mcdonalds.co.kr/kor/menu/listContent.do"
        self.detail_url = "https://www.mcdonalds.co.kr/kor/menu/detail.do"
        self.headers = {
            "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
            "User-Agent": "Mozilla/5.0"
        }

    # 특정 카테고리의 메뉴 데이터를 가져오는 메서드
    def fetch_menu_data(self, sub_category_seq, page_num):
        data = {
            "page": page_num,
            "sub_category_seq": sub_category_seq  # 서브 카테고리 번호를 포함한 데이터
        }
        try:
            # POST 요청을 통해 메뉴 데이터를 가져옴
            response = requests.post(self.url, headers=self.headers, data=data)
            response.raise_for_status()
            return response.json().get('list', [])  # 응답 데이터에서 'list' 항목을 반환
        except requests.exceptions.RequestException as e:
            print(f"API 요청 실패: {e}")  # 요청 실패 시 예외 처리
            return []

    # 메뉴의 상세 정보를 가져오는 메서드
    def fetch_menu_detail(self, menu_code, sub_category_seq):
        data = {"seq": menu_code, "page": 1, "sub_category_seq": sub_category_seq}  # 메뉴 코드와 서브 카테고리 정보
        try:
            # POST 요청을 통해 상세 메뉴 정보를 가져옴
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            # 메뉴 이름과 설명을 HTML에서 추출
            menu_name_tag = soup.find('h2', class_='ko')
            menu_desc_tag = soup.find('div', class_='desc')

            # 메뉴 이름과 설명이 없을 경우 기본 메시지 설정
            menu_name = menu_name_tag.get_text(strip=True) if menu_name_tag else "메뉴 이름을 찾을 수 없음"
            menu_name = re.sub(r'[™®]', '', menu_name)  # 특수 기호 제거
            
            menu_desc = menu_desc_tag.get_text(strip=True) if menu_desc_tag else "메뉴 설명을 찾을 수 없음"
            menu_desc = re.sub(r'[™®]', '', menu_desc)  # 특수 기호 및 판매 시간 관련 문구 제거
            menu_desc = re.sub(r'\*?판매\s*시간\s*[:：]?\s*[^\*]+', '', menu_desc).strip()

            return {"menu_name": menu_name, "menu_desc": menu_desc}
        except requests.exceptions.RequestException as e:
            print(f"상세 정보 API 요청 실패: {e}")
            return {}

    # 메뉴의 영양 정보를 가져오는 메서드
    def fetch_nutrition_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', class_='tableType01 nutrDesc')  # 영양 정보 테이블 찾기

            if table:
                # 테이블에서 데이터를 행 단위로 추출하여 리스트에 저장
                rows = table.find_all('tr')
                nutrition_info = [[col.text.strip() for col in row.find_all(['th', 'td'])] for row in rows]
                return nutrition_info
            else:
                return []
        except requests.exceptions.RequestException as e:
            print(f"영양 정보 API 요청 실패: {e}")
            return []

    # 메뉴의 알러지 정보를 가져오는 메서드
    def fetch_allergy_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            allergy_info_section = soup.find('div', class_='allerDesc')
            if allergy_info_section:
                # 알러지 정보에서 특수 기호 제거 후 반환
                allergy_info = allergy_info_section.get_text(strip=True)
                return re.sub(r'[™®]', '', allergy_info)
            else:
                return "알러지 정보 없음"
        except requests.exceptions.RequestException as e:
            return "알러지 정보 요청 실패"

    # 메뉴의 원산지 정보를 가져오는 메서드
    def fetch_origin_data(self, seq, sub_category_seq):
        data = {"seq": seq, "page": 1, "sub_category_seq": sub_category_seq}
        try:
            response = requests.post(self.detail_url, headers=self.headers, data=data)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            origin_info_section = soup.find('ul', class_='origin_info')
            if origin_info_section:
                # 원산지 정보에서 특수 기호 제거 후 반환
                origin_info = origin_info_section.get_text(strip=True)
                return re.sub(r'[™®]', '', origin_info)
            else:
                return "원산지 정보 없음"
        except requests.exceptions.RequestException as e:
            return "원산지 정보 요청 실패"

    # 텍스트에서 특수 문자를 제거하는 메서드
    def clean_text(self, text):
        cleaned_text = BeautifulSoup(text, "html.parser").get_text()  # HTML 태그 제거
        cleaned_text = re.sub(r'[™®]', '', cleaned_text)  # 특수 기호 제거
        return cleaned_text

    # 영양 정보를 포맷팅하는 메서드
    def format_nutrition_info(self, nutrition_data):
        if not nutrition_data:
            return "영양소 정보 없음"
        headers = nutrition_data[0]
        values = nutrition_data[1]
        formatted_nutrition = '\n'.join([f"{header}: {value}" for header, value in zip(headers, values) if header != "영양소"])
        return formatted_nutrition

    # 모든 메뉴 데이터를 하나의 파일로 저장하는 메서드
    def save_menus_to_file(self, all_menu_items_by_category):
        with open("/content/drive/MyDrive/Aiffel/Project/all_menu_data.txt", "w", encoding="utf-8") as f:
            for sub_category_seq, all_menu_items in all_menu_items_by_category.items():
                for item in all_menu_items:
                    kor_name = self.clean_text(item.get('kor_name', ''))
                    menu_code = item.get('seq', '')
                    menu_details = self.fetch_menu_detail(menu_code, sub_category_seq)
                    f.write(f"{kor_name}\n제품소개 : {menu_details['menu_desc']}\n")
                    nutrition_data = self.fetch_nutrition_data(menu_code, sub_category_seq)
                    formatted_nutrition = self.format_nutrition_info(nutrition_data)
                    f.write(f"영양소 정보:\n{formatted_nutrition}\n")
                    allergy_info = self.fetch_allergy_data(menu_code, sub_category_seq)
                    f.write(f"알러지 정보: {allergy_info}\n")
                    origin_info = self.fetch_origin_data(menu_code, sub_category_seq)
                    f.write(f"원산지 정보: {origin_info}\n\n")
        print("메뉴 데이터를 파일에 저장했습니다.")

# McDonaldsMenu 클래스 인스턴스 생성 및 실행
menu = McDonaldsMenu()
all_menu_items_by_category = {}

# 각 카테고리별로 메뉴 데이터를 가져와 저장
for sub_category_seq in range(1, 16):  # 카테고리 번호 1부터 15까지 반복
    all_menu_items = []
    for page_num in range(1, 6):  # 각 카테고리에서 5 페이지까지 데이터를 가져옴
        menu_data = menu.fetch_menu_data(sub_category_seq, page_num)
        if not menu_data:
            break  # 더 이상 데이터가 없으면 루프 중단
        all_menu_items.extend(menu_data)  # 데이터를 리스트에 추가
    all_menu_items_by_category[sub_category_seq] = all_menu_items  # 카테고리별로 데이터를 저장

menu.save_menus_to_file(all_menu_items_by_category)  # 모든 메뉴 데이터를 하나의 파일로 저장
